In [2]:
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import time
import os

In [3]:
# load data
Dtrain = np.loadtxt('Data/D_20.txt')
print(Dtrain.shape)

# splitting
xy  = np.random.permutation(int(Dtrain.shape[0]))
le  = int(Dtrain.shape[0]*0.3)
idt = xy[le:]
idv = xy[:le]

Dt = Dtrain[idt]
Dv = Dtrain[idv]

print(Dt.shape,Dv.shape)

(500000, 11)
(350000, 11) (150000, 11)


In [4]:
kBT   = 10
kBT_  = 20
print(kBT,kBT_)

n = 10
aa = np.array([-0.558223634633, 1.4417258418],dtype=np.float64)
bb = np.array([0.623499404931, 0.0280377585287],dtype=np.float64)

def construct_NN(x,nlayer,nnode):
    q = tf.contrib.layers.fully_connected(x,nnode,tf.nn.tanh,scope ="layer1")
    for i in range(1,nlayer):
        q = tf.contrib.layers.fully_connected(q,nnode,tf.nn.tanh,scope ="layer"+str(i+1))
    q = tf.contrib.layers.fully_connected(q,1,tf.nn.sigmoid,scope="output")
    dA2 = (x[:,0]-aa[0])**2+(x[:,1]-aa[1])**2
    dB2 = (x[:,0]-bb[0])**2+(x[:,1]-bb[1])**2
    IA = 0.5-0.5*tf.nn.tanh( 1000*(dA2-0.1**2)  )
    IB = 0.5-0.5*tf.nn.tanh( 1000*(dB2-0.1**2)  )
    return (1-IA)*(1-IB)*tf.reshape(q,[-1])+(1-IA)*IB

10 20


In [6]:
def SGD(D_t,D_v,nupdate,nlearn):
    print(np.shape(D_t),np.shape(D_v))
    saver.restore(sess,NNfile)
    vmin = sess.run(fn,{x:D_v[:,1:], coef:np.exp(-(1/kBT-1/kBT_)*D_v[:,0]) })
    imax = -1
    start = time.time()
    for i in range(nlearn):
        batch_idx = np.random.randint(D_t.shape[0], size=batch_size)
        batch = D_t[batch_idx,:]
        sess.run(train_step,{x:batch[:,1:], coef:np.exp(-(1/kBT-1/kBT_)*batch[:,0]) })
        if i % nupdate == 0 or i+1 == nlearn:
            cc = sess.run(fn,{x:D_v[:,1:], coef:np.exp(-(1/kBT-1/kBT_)*D_v[:,0]) })
            if cc<vmin:
                vmin = cc
                imax = i
                saver.save(sess, NNfile)
            c1 = sess.run(fn,{x:D_t[:,1:], coef:np.exp(-(1/kBT-1/kBT_)*D_t[:,0]) })
            print('step: %d, imax: %d, used time: %0.1f\t vmin: %0.3e, train: %0.3e, valid: %0.3e' 
                  %(i,imax,time.time()-start,vmin,c1,cc))
            start = time.time()
        if i-imax>3000:
            print('training stops!\n')
            break

In [7]:
tf.reset_default_graph()

# input
x = tf.placeholder(dtype=tf.float64, shape=[None,n])
coef = tf.placeholder(dtype=tf.float64, shape=[None,])

# q and gradients
q = construct_NN(x,2,20)
g = tf.gradients(q, x)[0]
fn = tf.reduce_mean( tf.reduce_sum(g**2,axis=1)*coef )

# train_step and initialization
train_step = tf.train.AdamOptimizer(0.001).minimize(fn)
init = tf.global_variables_initializer()
saver = tf.train.Saver(tf.trainable_variables())
sess = tf.Session()
sess.run(init)

In [8]:
kBT   = 10
kBT_  = 20
print(kBT,kBT_)

NNfile = "saved_models/Mueller_"+str(kBT)+"_"+str(kBT_)+"_220.ckpt"
saver.save(sess,NNfile)

batch_size = int(1e5)
SGD(Dt,Dv,100,50000)

sess.run(init)
SGD(Dt,Dv,1,1000)

10 20
(350000, 11) (150000, 11)
INFO:tensorflow:Restoring parameters from saved_models/Mueller_10_20_220.ckpt
step: 0, imax: 0, used time: 1.4	 vmin: 2.101e+03, train: 2.149e+03, valid: 2.101e+03
step: 100, imax: 100, used time: 22.3	 vmin: 1.028e+02, train: 1.097e+02, valid: 1.028e+02
step: 200, imax: 200, used time: 21.4	 vmin: 4.449e+01, train: 4.704e+01, valid: 4.449e+01
step: 300, imax: 300, used time: 20.9	 vmin: 1.947e+01, train: 2.031e+01, valid: 1.947e+01
step: 400, imax: 400, used time: 21.1	 vmin: 1.005e+01, train: 1.043e+01, valid: 1.005e+01
step: 500, imax: 500, used time: 22.0	 vmin: 6.084e+00, train: 6.285e+00, valid: 6.084e+00
step: 600, imax: 600, used time: 23.4	 vmin: 4.099e+00, train: 4.225e+00, valid: 4.099e+00
step: 700, imax: 700, used time: 23.7	 vmin: 3.009e+00, train: 3.094e+00, valid: 3.009e+00
step: 800, imax: 800, used time: 21.1	 vmin: 2.335e+00, train: 2.398e+00, valid: 2.335e+00
step: 900, imax: 900, used time: 19.8	 vmin: 1.890e+00, train: 1.936e+00, va

step: 8800, imax: 8800, used time: 19.3	 vmin: 4.126e-01, train: 4.108e-01, valid: 4.126e-01
step: 8900, imax: 8900, used time: 19.5	 vmin: 4.102e-01, train: 4.084e-01, valid: 4.102e-01
step: 9000, imax: 9000, used time: 19.7	 vmin: 4.073e-01, train: 4.058e-01, valid: 4.073e-01
step: 9100, imax: 9100, used time: 19.6	 vmin: 4.044e-01, train: 4.029e-01, valid: 4.044e-01
step: 9200, imax: 9200, used time: 20.6	 vmin: 4.006e-01, train: 3.996e-01, valid: 4.006e-01
step: 9300, imax: 9300, used time: 22.2	 vmin: 3.961e-01, train: 3.955e-01, valid: 3.961e-01
step: 9400, imax: 9400, used time: 21.9	 vmin: 3.896e-01, train: 3.900e-01, valid: 3.896e-01
step: 9500, imax: 9500, used time: 20.3	 vmin: 3.802e-01, train: 3.818e-01, valid: 3.802e-01
step: 9600, imax: 9600, used time: 21.7	 vmin: 3.632e-01, train: 3.670e-01, valid: 3.632e-01
step: 9700, imax: 9700, used time: 20.7	 vmin: 3.266e-01, train: 3.334e-01, valid: 3.266e-01
step: 9800, imax: 9800, used time: 21.6	 vmin: 2.524e-01, train: 2.601

step: 17500, imax: 17400, used time: 20.1	 vmin: 2.782e-02, train: 2.778e-02, valid: 2.784e-02
step: 17600, imax: 17400, used time: 22.4	 vmin: 2.782e-02, train: 2.778e-02, valid: 2.790e-02
step: 17700, imax: 17700, used time: 22.7	 vmin: 2.772e-02, train: 2.764e-02, valid: 2.772e-02
step: 17800, imax: 17800, used time: 22.7	 vmin: 2.755e-02, train: 2.757e-02, valid: 2.755e-02
step: 17900, imax: 17800, used time: 22.6	 vmin: 2.755e-02, train: 2.750e-02, valid: 2.756e-02
step: 18000, imax: 18000, used time: 22.7	 vmin: 2.751e-02, train: 2.741e-02, valid: 2.751e-02
step: 18100, imax: 18100, used time: 21.4	 vmin: 2.741e-02, train: 2.733e-02, valid: 2.741e-02
step: 18200, imax: 18200, used time: 20.5	 vmin: 2.738e-02, train: 2.726e-02, valid: 2.738e-02
step: 18300, imax: 18200, used time: 22.8	 vmin: 2.738e-02, train: 2.719e-02, valid: 2.738e-02
step: 18400, imax: 18400, used time: 22.8	 vmin: 2.726e-02, train: 2.710e-02, valid: 2.726e-02
step: 18500, imax: 18500, used time: 22.7	 vmin: 2

step: 26200, imax: 24100, used time: 24.4	 vmin: 2.602e-02, train: 2.521e-02, valid: 2.618e-02
step: 26300, imax: 24100, used time: 23.7	 vmin: 2.602e-02, train: 2.519e-02, valid: 2.622e-02
step: 26400, imax: 24100, used time: 21.8	 vmin: 2.602e-02, train: 2.525e-02, valid: 2.642e-02
step: 26500, imax: 24100, used time: 20.9	 vmin: 2.602e-02, train: 2.524e-02, valid: 2.631e-02
step: 26600, imax: 24100, used time: 20.8	 vmin: 2.602e-02, train: 2.518e-02, valid: 2.640e-02
step: 26700, imax: 24100, used time: 20.6	 vmin: 2.602e-02, train: 2.518e-02, valid: 2.641e-02
step: 26800, imax: 24100, used time: 23.0	 vmin: 2.602e-02, train: 2.519e-02, valid: 2.627e-02
step: 26900, imax: 24100, used time: 22.9	 vmin: 2.602e-02, train: 2.516e-02, valid: 2.636e-02
step: 27000, imax: 24100, used time: 23.0	 vmin: 2.602e-02, train: 2.514e-02, valid: 2.632e-02
step: 27100, imax: 24100, used time: 22.9	 vmin: 2.602e-02, train: 2.515e-02, valid: 2.647e-02
training stops!

(350000, 11) (150000, 11)
INFO:te

step: 82, imax: -1, used time: 0.7	 vmin: 2.602e-02, train: 2.538e-02, valid: 2.615e-02
step: 83, imax: -1, used time: 0.7	 vmin: 2.602e-02, train: 2.539e-02, valid: 2.617e-02
step: 84, imax: -1, used time: 0.7	 vmin: 2.602e-02, train: 2.538e-02, valid: 2.614e-02
step: 85, imax: -1, used time: 0.7	 vmin: 2.602e-02, train: 2.537e-02, valid: 2.612e-02
step: 86, imax: -1, used time: 0.8	 vmin: 2.602e-02, train: 2.535e-02, valid: 2.609e-02
step: 87, imax: -1, used time: 0.8	 vmin: 2.602e-02, train: 2.534e-02, valid: 2.605e-02
step: 88, imax: -1, used time: 0.7	 vmin: 2.602e-02, train: 2.534e-02, valid: 2.603e-02
step: 89, imax: 89, used time: 0.8	 vmin: 2.601e-02, train: 2.534e-02, valid: 2.601e-02
step: 90, imax: 90, used time: 0.8	 vmin: 2.601e-02, train: 2.534e-02, valid: 2.601e-02
step: 91, imax: 90, used time: 0.7	 vmin: 2.601e-02, train: 2.534e-02, valid: 2.602e-02
step: 92, imax: 90, used time: 0.7	 vmin: 2.601e-02, train: 2.533e-02, valid: 2.603e-02
step: 93, imax: 90, used time: 0

...

step: 543, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.529e-02, valid: 2.606e-02
step: 544, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.529e-02, valid: 2.607e-02
step: 545, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.529e-02, valid: 2.608e-02
step: 546, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.529e-02, valid: 2.608e-02
step: 547, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.528e-02, valid: 2.606e-02
step: 548, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.528e-02, valid: 2.605e-02
step: 549, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.528e-02, valid: 2.605e-02
step: 550, imax: 470, used time: 1.0	 vmin: 2.599e-02, train: 2.528e-02, valid: 2.605e-02
step: 551, imax: 470, used time: 1.0	 vmin: 2.599e-02, train: 2.528e-02, valid: 2.606e-02
step: 552, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.528e-02, valid: 2.606e-02
step: 553, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.528e-02, valid: 2.606e-02
step: 554,

step: 635, imax: 470, used time: 0.8	 vmin: 2.599e-02, train: 2.529e-02, valid: 2.618e-02
step: 636, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.530e-02, valid: 2.617e-02
step: 637, imax: 470, used time: 0.8	 vmin: 2.599e-02, train: 2.530e-02, valid: 2.615e-02
step: 638, imax: 470, used time: 0.8	 vmin: 2.599e-02, train: 2.530e-02, valid: 2.614e-02
step: 639, imax: 470, used time: 0.8	 vmin: 2.599e-02, train: 2.529e-02, valid: 2.614e-02
step: 640, imax: 470, used time: 0.9	 vmin: 2.599e-02, train: 2.528e-02, valid: 2.613e-02
step: 641, imax: 470, used time: 0.9	 vmin: 2.599e-02, train: 2.528e-02, valid: 2.613e-02
step: 642, imax: 470, used time: 0.9	 vmin: 2.599e-02, train: 2.527e-02, valid: 2.614e-02
step: 643, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.527e-02, valid: 2.613e-02
step: 644, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.527e-02, valid: 2.612e-02
step: 645, imax: 470, used time: 0.9	 vmin: 2.599e-02, train: 2.527e-02, valid: 2.611e-02
step: 646,

step: 727, imax: 470, used time: 0.9	 vmin: 2.599e-02, train: 2.527e-02, valid: 2.622e-02
step: 728, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.531e-02, valid: 2.630e-02
step: 729, imax: 470, used time: 0.8	 vmin: 2.599e-02, train: 2.535e-02, valid: 2.637e-02
step: 730, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.538e-02, valid: 2.640e-02
step: 731, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.538e-02, valid: 2.640e-02
step: 732, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.536e-02, valid: 2.637e-02
step: 733, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.533e-02, valid: 2.631e-02
step: 734, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.529e-02, valid: 2.625e-02
step: 735, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.527e-02, valid: 2.618e-02
step: 736, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.527e-02, valid: 2.615e-02
step: 737, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.528e-02, valid: 2.614e-02
step: 738,

step: 819, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.527e-02, valid: 2.621e-02
step: 820, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.526e-02, valid: 2.618e-02
step: 821, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.527e-02, valid: 2.616e-02
step: 822, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.527e-02, valid: 2.614e-02
step: 823, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.528e-02, valid: 2.613e-02
step: 824, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.528e-02, valid: 2.614e-02
step: 825, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.527e-02, valid: 2.614e-02
step: 826, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.526e-02, valid: 2.614e-02
step: 827, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.526e-02, valid: 2.614e-02
step: 828, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.526e-02, valid: 2.615e-02
step: 829, imax: 470, used time: 0.7	 vmin: 2.599e-02, train: 2.525e-02, valid: 2.616e-02
step: 830,

step: 911, imax: 878, used time: 0.6	 vmin: 2.599e-02, train: 2.529e-02, valid: 2.604e-02
step: 912, imax: 878, used time: 0.6	 vmin: 2.599e-02, train: 2.528e-02, valid: 2.604e-02
step: 913, imax: 878, used time: 0.6	 vmin: 2.599e-02, train: 2.527e-02, valid: 2.604e-02
step: 914, imax: 878, used time: 0.6	 vmin: 2.599e-02, train: 2.528e-02, valid: 2.607e-02
step: 915, imax: 878, used time: 0.6	 vmin: 2.599e-02, train: 2.528e-02, valid: 2.608e-02
step: 916, imax: 878, used time: 0.6	 vmin: 2.599e-02, train: 2.528e-02, valid: 2.608e-02
step: 917, imax: 878, used time: 0.6	 vmin: 2.599e-02, train: 2.528e-02, valid: 2.609e-02
step: 918, imax: 878, used time: 0.6	 vmin: 2.599e-02, train: 2.527e-02, valid: 2.609e-02
step: 919, imax: 878, used time: 0.6	 vmin: 2.599e-02, train: 2.527e-02, valid: 2.610e-02
step: 920, imax: 878, used time: 0.6	 vmin: 2.599e-02, train: 2.527e-02, valid: 2.611e-02
step: 921, imax: 878, used time: 0.6	 vmin: 2.599e-02, train: 2.527e-02, valid: 2.613e-02
step: 922,